## Organoid simulation (credit: Mason Hargrave, modif: Lucas Seninge)

This notebook is based on Mason's method to simulate organoids. It uses a stochastic model to determine which neurons will fire at t+1 based on neurons firing at t. See Mason's model description.

In [47]:
# Import modules
import numpy as np
np.random.seed(42)

Added inhibitory neurons to the total population: User can pick a proportion of inhibitory neurons which will be randomly pick in the neuron vector. </br>

In the case of inhibitory neurons, neuron  j firing at time  t will cause neuron  i to NOT fire at time  t+1. So we take the inverse of this probability so that in the product of _(1-pij)_ , inhibitory neurons firing will decrease the probability of neuron i firing. </br>

This simple model may need refinement as the biology of neuron inhibition is slightly different. </br>

Also, we added a refractory period: if a neuron was firing at t, he won't be able to fire at t+1, no matter what is the probability.

## Model Initialization

In [48]:
# Simulation parameters
# Number of neurons
neurons = 50

# Proportion of inhibitory neurons in the neuron population
prop_inhib = 0.2
# Generate random labels for inhibitory neurons / get labels for excitatory
inhib_idx = np.random.choice(neurons, int(prop_inhib*neurons))
excitatory_idx = np.array(list(set(range(neurons))-set(inhib_idx)))

# Probability matrix - initialize randomly
probability_matrix = np.random.rand(neurons,neurons)

# Neurons state - initialize randomly
initial_state = np.random.randint(2, size=neurons)
# Number of iterations - discret time steps (change to actual time?)
time = 100

In [49]:
# Preprocessing
# Scale-down probabilities by using index distance as a proxy for real distance
for (i,j) in np.ndindex(np.shape(probability_matrix)):
    power = abs(i-j)
    # Avoid self firing for now
    if power == 0:
        probability_matrix[i,j] = 0
    else:
        probability_matrix[i,j] = ((1/neurons)**power) * probability_matrix[i,j]

## Optimized version of next_state

In [50]:
def next_state(neurons, probability_matrix, previous_state):
    """
    Get new state t+1 of active/inactive neurons by using probabilities of firing.
    
    Args:
        neurons (int): number of neurons to be simulated.
        probability_matrix (np.array): probability of neuron j firing at t causing neuron i to fire at t+1.
        previous_state (np.array): vector of neuron states at t .
        
    Returns:
        next_state (np.array): vector of neuron states at t+1.
    """
    
    # Initialize values
    next_state = np.zeros(neurons)
    new_mat = 1-(probability_matrix*initial_state)
    
    new_mat = np.ma.array(new_mat, mask = False)
    # Get probability of firing - numpy optimization
    # Mask inhibitory neurons - get normal product ie excitatory neurons contribution
    new_mat.mask[:,inhib_idx] = True
    product = np.prod(new_mat, axis=1)
    # Unmask inhibitory and get inverse of product for inhibitory contribution
    new_mat.mask[:, inhib_idx] = False
    new_mat.mask[:, excitatory_idx] = True
    # Compute expect values pi as 1-full product
    expectation_values = 1-(product*(1./np.prod(new_mat, axis=1)))
    
    # Iterate on matrix dimensions
    for i in range(neurons):
        # Generate random number
        random = np.random.random(1)
        # Add OR statement so a neuron fired at time t can't fire at t+1 (Refractory period)
        if random > expectation_values[i] or previous_state[i]==1:
            next_state[i] = 0
        else:
            next_state[i] = 1

    return next_state

In [51]:
print('Indices of inhibitory neurons:',inhib_idx)
print(initial_state)
for t in range(time):
    initial_state = next_state(neurons, probability_matrix, initial_state)
    print(initial_state)

Indices of inhibitory neurons: [38 28 14 42  7 20 38 18 22 10]
[0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 0
 1 0 1 1 0 1 0 0 0 1 1 1 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

## Old version unoptimized

In [ ]:
def next_state(neurons, probability_matrix, previous_state):
    """
    Get new state t+1 of active/inactive neurons by using probabilities of firing.
    
    Args:
        neurons (int): number of neurons to be simulated.
        probability_matrix (np.array): probability of neuron j firing at t causing neuron i to fire at t+1.
        previous_state (np.array): vector of neuron states at t .
        
    Returns:
        next_state (np.array): vector of neuron states at t+1.
    """
    
    # Initialize values
    next_state = np.zeros(neurons)
    expectation_values = np.zeros(neurons)
    
    # Iterate on matrix dimensions
    for i in range(neurons):
        # initialize product for neuron i
        product = 1
        for j in range(neurons):
            # --Get probabilities of neuron i firing
            # if neuron j firing at time t is inhibitory, he will decrease the probability of i to fire at t+1 
            if j in inhib_idx:
                product *= (1-probability_matrix[i,j]*initial_state[j])
            # Otherwise it is an excitatory neuron and we apply normal formula
            else:
                product *= (1-probability_matrix[i,j]*initial_state[j])   
        # Get probability of firing
        expectation_values[i]=1-product
        
    print(expectation_values)
    # Get next step from new probability profile
    for i in range(neurons):
        random = np.random.random(1)
        # Add OR statement so a neuron fired at time t can't fire at t+1 (Refractory period)
        if random > expectation_values[i] or previous_state[i]==1:
            next_state[i] = 0
        else:
            next_state[i] = 1
            
    return next_state

print('Indices of inhibitory neurons:',inhib_idx)
print(initial_state)
for t in range(time):
    initial_state = next_state(neurons, probability_matrix, initial_state)
    #print(initial_state)